### SPRINT2

#### Testar e implementar a extração de dados

Exploraremos mais a fundo o Yahoo Finance a fim de realizar mais testes com os dados e atestar a sua confiabilidade e limitações

In [8]:
#importando a biblioteca do yahoo finance
import yfinance as yf

#importando bibliotecas para uso de datas
from datetime import datetime, timedelta

#importando blibliotecas para exibir gráficos para testes
import plotly.graph_objs as go

#importando biblioteca para medir o tempo de execução
import time

#importando biblioteca para trabalhar com dataframes
import pandas as pd

# Lista de países e intervalos de tempo
countries = ["Brazil", "United States"]
intervals = ["1d", "1wk", "1mo"]  # d - dia wk - semana mo - mês

#criando uma função para formatar a data
def format_date(dt, format="%Y-%m-%d"):
    return dt.strftime(format)

# função que cria um grafo para teste de consulta e tempo de retorno
def plotCandleStick(df, acao="ticket"):
    trace1 = {
        "x": df.index,
        "open": df["Open"],
        "close": df["Close"],
        "high": df["High"],
        "low": df["Low"],
        "type": "candlestick",
        "name": acao,
        "showlegend": False,
    }

    data = [trace1]
    layout = go.Layout()

    return go.Figure(data=data, layout=layout)



Com base na função plotCandleStick apresentada acima e um DataFrame local (df), realizaremos alguns testes:

1. Teste de Integridade dos Dados

Descrição: Verificar se os dados retornados pela API estão completos e corretos.

Implementação:

* Simular a captura de dados históricos de uma ação usando a API do Yahoo Finance.

* Verificar se todas as colunas esperadas (Open, Close, High, Low, etc.) estão presentes no DataFrame.

* Validar se os dados não possuem valores nulos ou incorretos (exemplo: valores negativos ou muito discrepantes).

In [4]:
# Criaremos uma função chamada test_integrity  e usaremos a ação da Apple (AAPL)
# como exemplo para verificar a integridade dos dados obtidos.
def test_integrity(df):
    expected_columns = ["Open", "Close", "High", "Low"]
    for col in expected_columns:
        if col not in df.columns:
            print(f"A coluna {col} não está presente no dataframe!")
        elif df[col].isnull().any():
            print(f"A coluna {col} contém valores nulos!")
        elif (df[col] < 0).any():
            print(f"A coluna {col} possui valores negativos!")
    print("Checagem da integridade dos dados conlcuída.")

# Exemplo de uso:
# Baixando os dados com frequência diária da ação da Apple entre 01/01/2023 e 31/12/2023
df = yf.download("AAPL", start="2023-01-01", end="2023-12-31", interval="1d")
test_integrity(df)

[*********************100%%**********************]  1 of 1 completed

Checagem da integridade dos dados conlcuída.


2. Teste de Confiabilidade da API

Descrição: Testar a confiabilidade da API em termos de tempo de resposta e consistência dos dados.

Implementação:

* Realizar múltiplas requisições para a mesma ação e período e comparar os resultados para verificar consistência.

* Medir o tempo de resposta para avaliar a performance da API.

In [ ]:
import time

# Criaremos uma função para testar a confiabilidade dos dados obtidos da API
def test_reliability(stock_symbol, start_date, end_date, interval):
    start_time = time.time()
    
    df1 = yf.download(stock_symbol, start=start_date, end=end_date, interval=interval)
    df2 = yf.download(stock_symbol, start=start_date, end=end_date, interval=interval)

    end_time = time.time()
    
    if df1.equals(df2):
        print("Os dados são consistentes através de múltiplas requisições.")
    else:
        print("Inconsistências foram encontradas nos dados para múltiplas requisições!")
    
    print(f"Tempo levado para a requisição: {end_time - start_time:.2f} seconds")

# Exemplo de uso:
# Usando a ação da Apple (AAPL) entre os dias 1 de janeiro de 2023 e
# 31 de dezembro de 2023, com intervalo diário (1d)
test_reliability("AAPL", "2023-01-01", "2023-12-31", "1d")

Utilizaremos a função abaixo para realizar 1000 conexões simultâneas a uma mesma ação

In [10]:
# Criaremos uma função para testar a confiabilidade dos dados obtidos da API
def test_reliability(stock_symbol, start_date, end_date, interval, num_requests=1000):
    dfs = []
    start_time = time.time()
    
    for i in range(num_requests):
        print(f"Requisição {i + 1}/{num_requests}")
        df = yf.download(stock_symbol, start=start_date, end=end_date, interval=interval, progress=False)
        dfs.append(df)

    end_time = time.time()

    # Verificando a consistência entre todos os DataFrames
    consistent = True
    for i in range(1, num_requests):
        if not dfs[0].equals(dfs[i]):
            print(f"Inconsistências encontradas entre df1 e df{i+1}!")
            consistent = False

    if consistent:
        print("Os dados são consistentes através de múltiplas requisições.")
    else:
        print("Inconsistências foram encontradas nos dados para múltiplas requisições!")
    
    print(f"Tempo total para {num_requests} requisições: {end_time - start_time:.2f} segundos")

# Exemplo de uso:
# Usando a ação da Apple (AAPL) entre os dias 1 de janeiro de 2023 e 31 de dezembro de 2023, com intervalo diário (1d)
test_reliability("AAPL", "2023-01-01", "2023-12-31", "1d")


Requisição 1/1000
Requisição 2/1000
Requisição 3/1000
Requisição 4/1000
Requisição 5/1000
Requisição 6/1000
Requisição 7/1000
Requisição 8/1000
Requisição 9/1000
Requisição 10/1000
Requisição 11/1000
Requisição 12/1000
Requisição 13/1000
Requisição 14/1000
Requisição 15/1000
Requisição 16/1000
Requisição 17/1000
Requisição 18/1000
Requisição 19/1000
Requisição 20/1000
Requisição 21/1000
Requisição 22/1000
Requisição 23/1000
Requisição 24/1000
Requisição 25/1000
Requisição 26/1000
Requisição 27/1000
Requisição 28/1000
Requisição 29/1000
Requisição 30/1000
Requisição 31/1000
Requisição 32/1000
Requisição 33/1000
Requisição 34/1000
Requisição 35/1000
Requisição 36/1000
Requisição 37/1000
Requisição 38/1000
Requisição 39/1000
Requisição 40/1000
Requisição 41/1000
Requisição 42/1000
Requisição 43/1000
Requisição 44/1000
Requisição 45/1000
Requisição 46/1000
Requisição 47/1000
Requisição 48/1000
Requisição 49/1000
Requisição 50/1000
Requisição 51/1000
Requisição 52/1000
Requisição 53/1000
Re

3. Teste de Limitações da API

Descrição: Verificar as limitações da API em termos de períodos de tempo suportados e granularidade dos dados.

Implementação:

Tentar capturar dados com intervalos de tempo pequenos (como 1m - 1 minuto) ou longos períodos históricos para ver se a API retorna os dados corretamente ou se há alguma limitação.

In [14]:
def test_api_limits(stock_symbol):  # sourcery skip: extract-duplicate-method
    try:
        # Teste com um intervalo muito pequeno (1 minuto)
        df_min = yf.download(stock_symbol, period="1d", interval="1m")
        print("Teste 1 - Dados recuperados com sucesso com um intervalo pequeno.")
        print(df_min.head(1))
        print(df_min.tail(1))
    except Exception as e:
        print(f"Erro no Teste 1 usando um intervalo pequeno: {e}")

    try:
        # Teste com um longo período histórico
        df_long = yf.download(stock_symbol, start="2000-01-01", end="2023-12-31", interval="1mo")
        print("Teste 2 - com longo período histórico concluído com sucesso.")
        print(df_long.head(1))
        print(df_long.tail(1))
    except Exception as e:
        print(f"Error no Teste 2 - usando um período longo: {e}")

# Exemplo de uso:
test_api_limits("AAPL")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Teste 1 - Dados recuperados com sucesso com um intervalo pequeno.
                                 Open        High         Low       Close  \
Datetime                                                                    
2024-07-30 09:30:00-04:00  219.039993  219.419693  218.660004  218.669998   

                            Adj Close   Volume  
Datetime                                        
2024-07-30 09:30:00-04:00  218.669998  1279103  
                                 Open        High         Low       Close  \
Datetime                                                                    
2024-07-30 15:59:00-04:00  219.320007  219.337006  218.619995  218.789993   

                            Adj Close  Volume  
Datetime                                       
2024-07-30 15:59:00-04:00  218.789993  892426  
Teste 2 - com longo período histórico concluído com sucesso.
                Open      High       Low     Close  Adj Close       Volume
Date                                       

4. Teste de exibição da API

Descrição: Verificar se a recuperação de dados histórios da Apple podem ser exibidos graficamente.

Implementação:

Capturar dados históricos para ver se são exibidos corretamente, para utilizaremos
como referência o segundo semestre de 2023

In [16]:
# Definindo as datas de início e fim
start_date = "2023-07-01"
end_date = "2023-12-31"

# Baixando os dados históricos da Apple (AAPL) usando yfinance
df = yf.download("AAPL", start=start_date, end=end_date, interval="1d")

# Plotando o gráfico candlestick
fig = plotCandleStick(df, acao="AAPL")

# Exibindo o gráfico no Jupyter Notebook
fig.show()

[*********************100%%**********************]  1 of 1 completed
